In [5]:
import pandas as pd
import numpy as np

In [6]:
df_train= pd.read_csv('mytrain')
df_train['name'] = df_train['first']+df_train['last']+df_train['dob']

In [7]:
df_test = pd.read_csv('mytest')

In [8]:
df_train.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,scale_city_pop,scale_age,scale_state_fe,scale_city_fe,scale_category_fe,scale_job_fe,scale_merchant_fe,scale_zip_fe,scale_amt,name
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,-0.280895,-0.892170,-0.405791,-0.026052,-1.557372,-0.184968,-1.478751,0.235727,-1.336859,JenniferBanks1988-03-09
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,-0.291960,-0.311130,-0.848021,1.491598,0.976031,0.581454,0.874048,1.541006,0.876702,StephanieGill1978-06-21
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,-0.278715,0.618533,-1.369674,-1.551706,-0.290420,-1.681313,-0.295716,-1.750851,1.394969,EdwardSanchez1962-01-19
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,...,-0.286040,0.328014,-1.127582,-1.561710,1.262023,-0.685558,1.050366,-1.763877,0.250940,JeremyWhite1967-01-12
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,-0.292125,-0.775962,-0.445406,-0.037057,-0.878387,-0.938566,-0.877754,0.221398,0.200532,TylerGarcia1986-03-28


check if one person how many cards. one person one card but many transaction

install torch_geometric


In [9]:
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(df_train, stratify=df_train['is_fraud'], train_size = 0.5)

In [10]:
'''nam = df_train.name.unique()
n= 0
for m in nam:
    df = df_train.loc[df_train['name'] == m]
    if df.cc_num.nunique() > 1:
        n+= 1
print(n)'''

"nam = df_train.name.unique()\nn= 0\nfor m in nam:\n    df = df_train.loc[df_train['name'] == m]\n    if df.cc_num.nunique() > 1:\n        n+= 1\nprint(n)"

In [11]:
import torch
import torch_geometric
from torch_geometric.data.storage import BaseStorage, NodeStorage, EdgeStorage
from torch_geometric.data import HeteroData

In [12]:
#df_train = df_train.groupby('cc_num')
#df_train = df_train.obj

In [13]:
import category_encoders as ce
encoder = ce.CountEncoder(cols="cc_num", normalize=True)
df_train["cc_num1"] = encoder.fit_transform(df_train["cc_num"])
df_test["cc_num1"] = encoder.transform(df_test["cc_num"])
encoder = ce.CountEncoder(cols="trans_num", normalize=True)
df_train["trans_num1"] = encoder.fit_transform(df_train["trans_num"])
df_test["trans_num1"] = encoder.transform(df_test["trans_num"])

In [14]:
data = HeteroData()

In [15]:
data['cc_num'].x = df_train.cc_num
data['cos_time'].x = df_train.trans_hour_cos
data['sin_time'].x = df_train.trans_hour_sin
data['amt'].x = df_train.scale_amt
data['merchant'].x = df_train.merchant_fe

In [16]:
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F

In [17]:
'''G = nx.DiGraph()
G.add_nodes_from(df_train['cc_num1'].unique(), type = 'cc_num1')
G.add_nodes_from(df_train['merchant_fe'].unique(), type = 'merchant_fe')
G.add_nodes_from(df_train['trans_num1'], type = 'trans_num1')'''


"G = nx.DiGraph()\nG.add_nodes_from(df_train['cc_num1'].unique(), type = 'cc_num1')\nG.add_nodes_from(df_train['merchant_fe'].unique(), type = 'merchant_fe')\nG.add_nodes_from(df_train['trans_num1'], type = 'trans_num1')"

In [18]:
'''a = df_train.drop_duplicates(subset='name')
for _, row in a.iterrows():
    scale_age = row['scale_age']
    scale_job_fe = row['scale_job_fe']
    attrs = {row['cc_num1']:{'scale_age': scale_age, 'scale_job_fe': scale_job_fe}}
    nx.set_node_attributes(G, attrs)'''
    

"a = df_train.drop_duplicates(subset='name')\nfor _, row in a.iterrows():\n    scale_age = row['scale_age']\n    scale_job_fe = row['scale_job_fe']\n    attrs = {row['cc_num1']:{'scale_age': scale_age, 'scale_job_fe': scale_job_fe}}\n    nx.set_node_attributes(G, attrs)"

In [19]:
'''for _, row in df_train.iterrows():
  scale_amt = row['scale_amt']
  trans_hour_sin = row['trans_hour_sin']
  trans_hour_cos = row['trans_hour_cos']
  scale_state_fe = row['scale_state_fe']
  scale_category_fe = row['scale_category_fe']
  is_fraud = row['is_fraud']
  attrs = {row['trans_num']:{'scale_amt' : scale_amt,
                              'trans_hour_sin' : trans_hour_sin,
                              'trans_hour_cos' : trans_hour_cos,
                              'scale_state_fe' : scale_state_fe,
                              'scale_category_fe' : scale_category_fe}}
  nx.set_node_attributes(G, attrs)
  G.add_edge(row['cc_num1'], row['trans_num1'])
  G.add_edge(row['trans_num1'], row['merchant_fe'])'''

"for _, row in df_train.iterrows():\n  scale_amt = row['scale_amt']\n  trans_hour_sin = row['trans_hour_sin']\n  trans_hour_cos = row['trans_hour_cos']\n  scale_state_fe = row['scale_state_fe']\n  scale_category_fe = row['scale_category_fe']\n  is_fraud = row['is_fraud']\n  attrs = {row['trans_num']:{'scale_amt' : scale_amt,\n                              'trans_hour_sin' : trans_hour_sin,\n                              'trans_hour_cos' : trans_hour_cos,\n                              'scale_state_fe' : scale_state_fe,\n                              'scale_category_fe' : scale_category_fe}}\n  nx.set_node_attributes(G, attrs)\n  G.add_edge(row['cc_num1'], row['trans_num1'])\n  G.add_edge(row['trans_num1'], row['merchant_fe'])"

In [20]:
'''k = []
for i in range(1000):
    a = df_train['cc_num'][i]
    b = df_train['merchant'][i]
    c = df_train['trans_num'][i]
    k.append(a)
    k.append(b)
    k.append(c)
nx.spring_layout(G.subgraph(k))
pos = nx.spring_layout(G.subgraph(k), scale=20, k=3/np.sqrt(G.subgraph(k).order()))
d = dict(G.subgraph(k).degree)
nx.draw(G.subgraph(k), pos, 
        nodelist=d, 
        node_size=[d[k]*30 for k in d])'''

"k = []\nfor i in range(1000):\n    a = df_train['cc_num'][i]\n    b = df_train['merchant'][i]\n    c = df_train['trans_num'][i]\n    k.append(a)\n    k.append(b)\n    k.append(c)\nnx.spring_layout(G.subgraph(k))\npos = nx.spring_layout(G.subgraph(k), scale=20, k=3/np.sqrt(G.subgraph(k).order()))\nd = dict(G.subgraph(k).degree)\nnx.draw(G.subgraph(k), pos, \n        nodelist=d, \n        node_size=[d[k]*30 for k in d])"

In [21]:
import networkx as nx
import numpy as np
import torch
from torch_geometric.utils.convert import from_networkx

In [74]:
data = HeteroData()

In [77]:
a = df_train.drop_duplicates(subset='name')
pos_dict_cc = {}
i = 0
for _, row in a.iterrows():
    pos_dict_cc[row['cc_num']] = i
    i+=1
b = df_train.drop_duplicates(subset='merchant')
pos_dict_merchant = {}
ii = 0
for _ , row in b.iterrows():
    pos_dict_merchant[row['merchant']] = ii
    ii += 1

list1 = []
list2 = []
list3 = []
iii=0
for _, row in df_train.iterrows():
    list1.append(pos_dict_cc[row['cc_num']])
    list2.append(iii)
    list3.append(pos_dict_merchant[row['merchant']])
    iii+=1
    
edge_cc = torch.tensor([list1, list2])
edge_merchant = torch.tensor([list2, list3])


In [187]:
import torch
a = df_train.drop_duplicates(subset='name')
pos_dict_cc = {}
i = 0
for _, row in a.iterrows():
    pos_dict_cc[row['cc_num1']] = i
    i+=1
b = df_train.drop_duplicates(subset='merchant')
pos_dict_merchant = {}
ii = 0
for _ , row in b.iterrows():
    pos_dict_merchant[row['merchant']] = ii
    ii += 1

list1 = []
list2 = []
list3 = []
iii=0
for _, row in df_train.iterrows():
    list1.append(pos_dict_cc[row['cc_num1']])
    list2.append(iii)
    list3.append(pos_dict_merchant[row['merchant']])
    iii+=1
    
edge_cc = torch.tensor([list1, list2])
edge_merchant = torch.tensor([list2, list3])
def df_to_tensor(x: pd.DataFrame):
    return torch.from_numpy(x.values).to(torch.device('cpu'),dtype=torch.float)

dictionary = dict({'cc_num1':0,'scale_age':1,'scale_job_fe':2,'trans_num1':3,'trans_hour_cos':4,'trans_hour_sin':5,'scale_amt':6,'scale_category_fe':7, 'scale_state_fe':8,'merchant_fe':9})
df_cc_num = a[['scale_age','scale_job_fe']]
df_trans_num = df_train[['trans_hour_cos','trans_hour_sin','scale_amt','scale_category_fe', 'scale_state_fe']]
df_merchant = b[['merchant_fe']]
tensor_cc_num = df_to_tensor(df_cc_num)
tensor_tran_num = df_to_tensor(df_trans_num)
tensor_merchant = df_to_tensor(df_merchant)

data = HeteroData()
data['cc'].x = tensor_cc_num
data['trans'].x = tensor_tran_num
data['merchant'].x = tensor_merchant

In [188]:

data['cc','pay','trans'].edge_index = edge_cc
data['trans', 'obtain', 'merchant'].edge_index = edge_merchant

In [151]:
data.metadata

<bound method HeteroData.metadata of HeteroData(
  cc={ x=[976, 2] },
  trans={ x=[641489, 5] },
  merchant={ x=[693, 1] },
  (cc, pay, trans)={ edge_index=[2, 641489] },
  (trans, obtain, merchant)={ edge_index=[2, 641489] }
)>

In [127]:
'''user_dict = {}
for _, row in df_train.iterrows():
    cc = row['cc_num1']
    k = user_dict.get(cc)
    if k is None:
        user_dict[cc] = row['is_fraud']
        continue
    if k == 0:
        user_dict[cc] = row['is_fraud']
list5 = []
for '''
        


"user_dict = {}\nfor _, row in df_train.iterrows():\n    cc = row['cc_num1']\n    k = user_dict.get(cc)\n    if k is None:\n        user_dict[cc] = row['is_fraud']\n        continue\n    if k == 0:\n        user_dict[cc] = row['is_fraud']\nlist5 = []\nfor "

In [189]:
result = df_to_tensor(df_train[['is_fraud']]).tolist()
data['trans'].y = result

In [153]:
data.metadata

<bound method HeteroData.metadata of HeteroData(
  cc={ x=[976, 2] },
  trans={
    x=[641489, 5],
    y=[641489]
  },
  merchant={ x=[693, 1] },
  (cc, pay, trans)={ edge_index=[2, 641489] },
  (trans, obtain, merchant)={ edge_index=[2, 641489] }
)>

In [190]:
import torch_geometric.transforms as T
data = T.ToUndirected()(data)
data = T.NormalizeFeatures()(data)

In [191]:
data1 = T.RandomNodeSplit(split='train_rest')(data)
data1

HeteroData(
  cc={ x=[976, 2] },
  trans={
    x=[641489, 5],
    y=[641489],
    train_mask=[641489],
    val_mask=[641489],
    test_mask=[641489]
  },
  merchant={ x=[693, 1] },
  (cc, pay, trans)={ edge_index=[2, 641489] },
  (trans, obtain, merchant)={ edge_index=[2, 641489] },
  (trans, rev_pay, cc)={ edge_index=[2, 641489] },
  (merchant, rev_obtain, trans)={ edge_index=[2, 641489] }
)

In [197]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, to_hetero

class GNN(torch.nn.Module):
    def __init__(self, hidden, out):
        super().__init__()
        self.conv1 = SAGEConv((-1,-1), hidden)
        self.conv2 = SAGEConv((-1,-1), out)

    def forward(self, x, edge_index):
        x = torch.relu(self.conv1(x, edge_index)) 
        x = self.conv2(x, edge_index)
        return x
model = GNN(hidden= 64, out= 2)
model = to_hetero(model, data1.metadata(), aggr= 'sum')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import ModuleList
from torch_geometric.nn import Linear, SAGEConv

def weighted_cross_entropy(out, target, mask=None, weight=None):
    loss = F.cross_entropy(out, target, reduction="none")

    if weight is not None:
        weight = target * weight[1] + (1 - target) * weight[0]
        weight *= mask
        weight *= mask.sum() / weight.sum()
        loss *= weight

    loss *= mask
    loss = loss.sum() / mask.sum()
    return loss


'''class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, num_layers):
        super().__init__()
        assert num_layers > 1
        self.convs = ModuleList([SAGEConv((-1, -1), hidden_channels) for _ in range(num_layers)])

    def forward(self, x, edge_index):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i != len(self.convs) - 1:
                x = F.relu(x)
        return x'''


class Model(torch.nn.Module):
    def __init__(
        self,
        hetero_gnn,
        embedding_size,
        out_channels,
        node_types,
        num_nodes_per_type,
        class_weight=None,
        full_batch=False,
    ):
        super().__init__()
        self.hetero_gnn = hetero_gnn
        self.embedding = nn.ModuleDict(
            {
                node_type: nn.Embedding(num_nodes_per_type[node_type], embedding_size)
                for node_type in node_types
                if node_type != "trans"
            }
        )
        self.linear = Linear(-1, out_channels)
        self.node_types = node_types
        self.full_batch = full_batch
        self.class_weight = class_weight

    def forward(self, x_dict, edge_index_dict, batch_size=None, n_id_dict=None, target=None, mask=None):
        for node_type in self.node_types:
            if node_type != "trans":
                if self.full_batch:
                    x_dict[node_type] = self.embedding[node_type]
                else:
                    assert n_id_dict is not None, "If using a sampled batch, `n_id_dict` must be provided."
                    x_dict[node_type] = self.embedding[node_type](n_id_dict[node_type])

        x_dict = self.hetero_gnn(x_dict, edge_index_dict)
        out = self.linear(x_dict["trans"])
        if self.training:
            if not self.full_batch:
                assert batch_size is not None, "If using a sampled batch, `batch_size` must be provided."
                mask = (target * 0).bool()
                mask[:batch_size] = 1
            loss = weighted_cross_entropy(out, target, mask, self.class_weight)
            return out, loss
        return out

In [198]:
data.validate()

True

In [146]:


with torch.no_grad():
    out = model(data1.x_dict, data1.edge_index_dict)

In [196]:
from poptorch_geometric import FixedSizeOptions

torch.manual_seed(42)

fixed_size_options = FixedSizeOptions.from_loader(train_loader)
fixed_size_options

AttributeError: 'Tensor' object has no attribute 'y'

In [199]:
import random

import networkx as nx
from matplotlib import pyplot as plt
from torch_geometric.utils import to_networkx

# Convert to homogeneous
data_homogeneous = data.to_homogeneous()
g = to_networkx(data_homogeneous)
# Use node types as colour map
colour_map = data_homogeneous.node_type

pos = nx.spring_layout(g)

# Split the nodes by node type and add some randomness to separate the nodes
for i in range(0, len(colour_map)):
    if colour_map[i] != 0:
        pos[i][0] += np.cos(colour_map[i] / 2) * 10 + random.randint(-1, 1)
        pos[i][1] += np.sin(colour_map[i] / 2) * 10 + random.randint(-1, 1)
    else:
        pos[i][0] += random.randint(-3, 3)
        pos[i][1] += random.randint(-3, 3)

nx.draw_networkx(g, pos=pos, node_color=colour_map * 40, cmap=plt.cm.tab20)
plt.show()

KeyboardInterrupt: 

In [201]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x_dict, data.edge_index_dict)
    mask = data['trans'].train_mask
    loss = F.cross_entropy(out['trans'][mask], data['trans'].y[mask])
    loss.backward()
    optimizer.step()
    return float(loss)
train()

TypeError: only integer tensors of a single element can be converted to an index

: 